In [1]:
import pandas as pd
import matplotlib as plt
import numpy as np

from utils import (add_new_col, add_zero_right, col_replace,
                   col_upper, col_upper_first_letter, df_astype,
                   df_subtract_string, drop_col, drop_duplicates_keep_first,
                   drop_row, rename_col, str_extract, str_strip)

## Contexto

A equipe de produtos da Geofusion tem recebido feedbacks da equipe comercial e eles dizem que alguns clientes do setor de alimentação (restaurantes, pizzarias, bares, etc.) estão procurando soluções que os ajudem a entender melhor seus concorrentes.

Eles gostariam de saber:
- qual é a faixa de preço praticada pelos concorrentes
- como é o fluxo de pessoas nesses locais
- qual é a população e a densidade demográfica dos bairros onde os concorrentes estão

---

## Carregamento dos dados

Sugestão de estrutura. Utilize as células abaixo para carregar os dados e tratá-los na memória.

São 4 arquivos:
* [Bairros](https://s3.sa-east-1.amazonaws.com/geofusion-data-engineer-files/bairros.csv)
  * Contém dados sobre bairros com código do município.
* [Concorrentes](https://s3.sa-east-1.amazonaws.com/geofusion-data-engineer-files/concorrentes.csv)
  * Contém dados sobre os concorrentes do meu estabelecimento.
* [Eventos de Fluxo](https://s3.sa-east-1.amazonaws.com/geofusion-data-engineer-files/eventos_de_fluxo.csv.gz)
  * Contém dados de eventos de visitas aos concorrentes com código do cliente, data da visita e código do concorrente.
* [População](https://s3.sa-east-1.amazonaws.com/geofusion-data-engineer-files/populacao.json)
  * Contém dados sobre a população dos bairros.

Os arquivos também constam na pasta `./dados` desse projeto.

In [2]:
bairros_df = pd.read_csv('dados/bairros.csv')

In [3]:
concorrentes_df = pd.read_csv('dados/concorrentes.csv')

In [4]:
populacao_df = pd.read_json('dados/populacao.json')

In [5]:
eventos_fluxo_df = pd.read_csv('dados/eventos_de_fluxo.csv.gz', compression='gzip')

In [6]:
# Implementação...
# bairros_df.info()
# concorrentes_df.info()
# populacao_df.info()
# eventos_fluxo_df.info()

### Modelagem

* Prepare os dados e os tipos corretos. Não assuma nada no dataset que você recebeu.
* Modele uma estrutura (utilizando joins e merges) no pandas utilizando o Notebook em anexo. Essa estrutura precisa ser capaz de responder as seguintes perguntas:
  * qual é a faixa de preço praticada pelos concorrentes
  * como é o fluxo de pessoas nesses locais (média por dia, maximo e minimo por dia, etc)
  * qual é a população e a densidade demográfica dos bairros onde os concorrentes estão
* Sugira um schema de saída, com nome das colunas, tipos dos dados e uma sugestão de tipo de arquivo (CSV, Parquet, JSON, etc) e justifique. 
* Exemplifique com algumas queries em SQL como seria possivel nossos analistas extrairem as informações acima.

#### Entrega
* **Entregue o relatório acima em qualquer formato (Word, Markdown, etc).**
* **Comprima seu projeto com tudo que você utilizou e envie para nós.**
* **[plus] Comente sobre possíveis dificuldades que você tenha encontrado ao realizar o case.**

In [7]:
# Implementação...
bairros_df_astype = {
    "codigo": 'Int64',
    "nome": str,
    "municipio": str,
    "uf": str,
    "area": 'Float64'
}

bairros_df = df_astype(bairros_df, bairros_df_astype)
bairros_df = col_upper(bairros_df, "uf")

In [8]:
concorrentes_df = add_new_col(concorrentes_df, 5, "cep", None)

concorrentes_df_astype = {
    "codigo": 'Int64',
    "nome": str,
    "categoria": str,
    "faixa_preco": 'Int64',
    "endereco": str,
    "cep": str,
    "municipio": str,
    "uf": str,
    "codigo_bairro": 'Int64'
}

concorrentes_df = df_astype(concorrentes_df, concorrentes_df_astype)
concorrentes_df = str_extract(concorrentes_df, "endereco", "cep", r'(\d{5}-?\d{3}?)')
concorrentes_df = add_zero_right(concorrentes_df, "cep", 8)
concorrentes_df = col_upper(concorrentes_df, "uf")
concorrentes_df = col_upper_first_letter(concorrentes_df, "nome")
concorrentes_df = col_replace(concorrentes_df, r'(\d{5})(\d{3})', r'\1-\2', "cep")
concorrentes_df = col_replace(concorrentes_df, r'(\d{5}-?\d{3})?', "", "endereco")
concorrentes_df = df_subtract_string(concorrentes_df, "municipio", "endereco")
concorrentes_df = df_subtract_string(concorrentes_df, "uf", "endereco")
concorrentes_df = col_replace(concorrentes_df, r'[,]$', "", "endereco")
concorrentes_df = str_strip(concorrentes_df, "endereco")
concorrentes_df = col_replace(concorrentes_df, '', np.NaN, "endereco")
concorrentes_df = col_replace(concorrentes_df, ['nan'], [None])
concorrentes_df = drop_row(concorrentes_df, ["municipio"])

In [9]:
populacao_df_astype = {
    "codigo": 'Int64',
    "populacao": 'Float64'
}

populacao_df = df_astype(populacao_df, populacao_df_astype)
populacao_df = drop_row(populacao_df, ["populacao"])

In [10]:
eventos_fluxo_df[["datetime"]] = eventos_fluxo_df[["datetime"]].apply(lambda x: pd.to_datetime(x, format='ISO8601'))

eventos_fluxo_df_astype = {
    "codigo": str,
    "datetime": 'datetime64[ns]',
    "codigo_concorrente": 'Int64'
}

eventos_fluxo_df = df_astype(eventos_fluxo_df, eventos_fluxo_df_astype)
eventos_fluxo_df = drop_duplicates_keep_first(eventos_fluxo_df)

* Qual é a faixa de preço praticada pelos concorrentes?

In [11]:
faixa_preco_df = concorrentes_df.join(
    bairros_df.set_index('codigo'), how="inner", on="codigo_bairro", rsuffix="_bairro"
)

faixa_preco_df = drop_col(faixa_preco_df, ["municipio_bairro", "uf_bairro", "cep"])
faixa_preco_df = rename_col(faixa_preco_df, {'codigo': 'codigo_concorrente', 'nome': 'nome_concorrente'})

faixa_preco_df = faixa_preco_df.groupby(["codigo_bairro", "nome_bairro", "municipio", "uf", "categoria"])["faixa_preco"].agg(["max", "min", "mean"])

faixa_preco_df.head(2)

max   
codigo_bairro nome_bairro municipio uf categoria                                     
35095064      Centro      Campinas  SP Bar, Dance & Night Club                   4  \
                                       Brazilian Restaurant, Bar, Bar & Grill    3   

                                                                               min   
codigo_bairro nome_bairro municipio uf categoria                                     
35095064      Centro      Campinas  SP Bar, Dance & Night Club                   4  \
                                       Brazilian Restaurant, Bar, Bar & Grill    3   

                                                                               mean  
codigo_bairro nome_bairro municipio uf categoria                                     
35095064      Centro      Campinas  SP Bar, Dance & Night Club                  4.0  
                                       Brazilian Restaurant, Bar, Bar & Grill   3.0

* Como é o fluxo de pessoas nesses locais (média por dia, maximo e minimo por dia, etc)?

In [12]:
fluxo_pessoas_df = pd.merge(
    concorrentes_df,
    eventos_fluxo_df,
    how="inner",
    left_on="codigo",
    right_on="codigo_concorrente",
    suffixes=('_x', '_fluxo')
).join(
    bairros_df.set_index('codigo'), how="inner", on="codigo_bairro", rsuffix="_bairro"
)

fluxo_pessoas_df = drop_col(fluxo_pessoas_df, ["codigo_x", "municipio_bairro", "uf_bairro", "cep"])

fluxo_pessoas_df["date"] = fluxo_pessoas_df["datetime"].dt.normalize()

fluxo_pessoas_df_count = fluxo_pessoas_df.groupby(["codigo_bairro", "nome_bairro", "municipio", "uf", "categoria", "date"])["date"].agg(["count"])
fluxo_pessoas_df_count = fluxo_pessoas_df_count.groupby(["codigo_bairro", "nome_bairro", "municipio", "uf", "categoria"])["count"].agg(["max", "min", "mean"])

fluxo_pessoas_df_count.head(2)

max   
codigo_bairro nome_bairro municipio uf categoria                                     
35095064      Centro      Campinas  SP Bar, Dance & Night Club                  42  \
                                       Brazilian Restaurant, Bar, Bar & Grill   44   

                                                                               min   
codigo_bairro nome_bairro municipio uf categoria                                     
35095064      Centro      Campinas  SP Bar, Dance & Night Club                   1  \
                                       Brazilian Restaurant, Bar, Bar & Grill    1   

                                                                                    mean  
codigo_bairro nome_bairro municipio uf categoria                                          
35095064      Centro      Campinas  SP Bar, Dance & Night Club                 17.147059  
                                       Brazilian Restaurant, Bar, Bar & Grill  23.264706

* Qual é a população e a densidade demográfica dos bairros onde os concorrentes estão?

In [13]:
densidade_demo_df = bairros_df.join(
    populacao_df.set_index('codigo'),
    on="codigo",
    how="inner",
    rsuffix="_right"
).merge(
    concorrentes_df, how="inner", left_on="codigo", right_on="codigo_bairro"
)

densidade_demo_df['densidade_demo'] = densidade_demo_df['populacao'] / densidade_demo_df['area']

densidade_demo_df = drop_col(densidade_demo_df, ["codigo_x", "uf_y", "municipio_y"])
densidade_demo_df = rename_col(densidade_demo_df, {
    'nome_x': 'bairro',
    'municipio_x': 'municipio',
    'uf_x': 'uf',
    'nome_y': 'nome',
    'codigo_y': 'codigo',
})

densidade_demo_df = densidade_demo_df[["bairro", "municipio", "uf", "populacao", "densidade_demo"]]

densidade_demo_df = drop_duplicates_keep_first(densidade_demo_df)

densidade_demo_df.head(2)

,bairro,municipio,uf,populacao,densidade_demo
0,Observatório,Valinhos,SP,8717.0,128.18948
14,Rp 6-24,Hortolândia,SP,5764.0,5871.040816


Sugira um schema de saída, com nome das colunas, tipos dos dados e uma sugestão de tipo de arquivo (CSV, Parquet, JSON, etc) e justifique.

In [14]:
df = pd.merge(
    concorrentes_df,
    eventos_fluxo_df,
    how="inner",
    left_on="codigo",
    right_on="codigo_concorrente"
).join(
    populacao_df.set_index('codigo'), how="inner", on="codigo_bairro"
).join(
    bairros_df.set_index('codigo'), how="inner", on="codigo_bairro", rsuffix="_right"
)

df["date"] = df["datetime"].dt.normalize()
df['densidade_demo'] = df['populacao'] / df['area']

df = drop_col(df, ["codigo_x", "cep", "municipio_right", "uf_right"])
df = rename_col(df, {
    'codigo_y': 'codigo_fluxo',
    'nome': 'nome_concorrente',
    'nome_right': 'nome_bairro'
})

df = df[['codigo_concorrente', 'nome_concorrente', 'categoria', 'faixa_preco', 'endereco', 'municipio',
        'uf', 'codigo_bairro', 'nome_bairro', 'populacao', 'area', 'densidade_demo', 'codigo_fluxo',
        'datetime', 'date']]

Exemplo de como ficaria:

In [15]:
df.head(1)

,codigo_concorrente,nome_concorrente,categoria,faixa_preco,endereco,municipio,uf,codigo_bairro,nome_bairro,populacao,area,densidade_demo,codigo_fluxo,datetime,date
0,1806610362988069,Restaurante Jangada Dom Pedro,Restaurant,2,shopping D.Pedro,Campinas,SP,35095070,Taquaral,62735.0,13.1911,4755.858116,l5tFvFcOOEISPofKxd6hQqGsMM0n49jP96SOoTifcxJUOD...,2017-07-17 15:22:31.935,2017-07-17


Schema sugerido:

In [16]:
schema_astype = {
    "codigo_concorrente": 'Int64',
    "nome_concorrente": str,
    "categoria": str,
    "faixa_preco": 'Int64',
    "endereco": str,
    "municipio": str,
    "uf": str,
    "codigo_bairro": 'Int64',
    "nome_bairro": str,
    "populacao": 'Float64',
    "area": 'Float64',
    "densidade_demo": 'Float64',
    "codigo_fluxo": str,
    "datetime": 'datetime64[ns]',
    "date": 'datetime64[ns]',
}

df = df_astype(df, schema_astype)

Tipo de arquivo e justificativa:

Usaria o formato de arquivo de saída em Parquet. Por ser um formato de arquivo que possui um tempo de leitura rápido. É possível evoluir o schema. Os dados são armazenados em coluna, o que acabando sendo interessante quando se possui muitas colunas, visto que não é preciso analisar todo o arquivo, é possível verificar apenas os metadados, e selecionar apenas as colunas que o usuário deseja, lendo apenas partes da informação, o que economiza tempo e custo. O tamanho do armazenamento é menor em relação ao CSVs por exemplo.

## Saída (output)

Persista no disco um arquivo no disco de acordo com sua sugestão.

In [17]:
# Implementação...
df.to_parquet('saida/dataset_final.parquet')

#### Exemplifique com algumas queries em SQL como seria possivel nossos analistas extrairem as informações acima

OBS: As queries foram escritas em SQL, o pandas foi utilizado apenas para teste e validação
OBS 02: Foi colocado um limit nas queries apenas para reduzir a quantidade de dados

In [18]:
from pandasql import sqldf

Qual é a faixa de preço praticada pelos concorrentes?

In [19]:
query_01 = """
    SELECT
        "codigo_bairro", "nome_bairro", "municipio", "uf", "categoria", MAX("faixa_preco") as max_faixa_preco, MIN("faixa_preco") as min_faixa_preco, AVG("faixa_preco") as media_faixa_preco
    FROM
        df
    GROUP BY
        "codigo_bairro", "nome_bairro", "municipio", "uf", "categoria"
    LIMIT 3
"""
sqldf(query_01, globals())

,codigo_bairro,nome_bairro,municipio,uf,categoria,max_faixa_preco,min_faixa_preco,media_faixa_preco
0,35095064,Centro,Campinas,SP,"Bar, Dance & Night Club",4,4,4.0
1,35095064,Centro,Campinas,SP,"Brazilian Restaurant, Bar, Bar & Grill",3,3,3.0
2,35095064,Centro,Campinas,SP,"Brewery, Bar, Restaurant",2,2,2.0


Como é o fluxo de pessoas nesses locais (média por dia, maximo e minimo por dia, etc)?

In [20]:
query_02 = """
    SELECT
        "codigo_bairro"
        , "nome_bairro"
        , "municipio"
        , "uf"
        , "categoria"
        , MAX("count_date") as max_por_dia
        , MIN("count_date") as min_por_dia
        , AVG("count_date") as media_por_dia
    FROM
        (
        SELECT
            "codigo_bairro", "nome_bairro", "municipio", "uf", "categoria", DATE("date") as date, COUNT("date") as count_date
        FROM
            df
        GROUP BY
            "codigo_bairro", "nome_bairro", "municipio", "uf", "categoria", "date"
    )
    GROUP BY
        "codigo_bairro", "nome_bairro", "municipio", "uf", "categoria"
    LIMIT 3
"""
sqldf(query_02, globals())

,codigo_bairro,nome_bairro,municipio,uf,categoria,max_por_dia,min_por_dia,media_por_dia
0,35095064,Centro,Campinas,SP,"Bar, Dance & Night Club",42,1,17.147059
1,35095064,Centro,Campinas,SP,"Brazilian Restaurant, Bar, Bar & Grill",44,1,23.264706
2,35095064,Centro,Campinas,SP,"Brewery, Bar, Restaurant",47,1,20.250000


Qual é a população e a densidade demográfica dos bairros onde os concorrentes estão?

In [21]:
query_03 = """
    SELECT DISTINCT
        "codigo_bairro", "nome_bairro", "municipio", "uf", "populacao", "densidade_demo"
    FROM
        df
    LIMIT 3
"""
sqldf(query_03, globals())

,codigo_bairro,nome_bairro,municipio,uf,populacao,densidade_demo
0,35095070,Taquaral,Campinas,SP,62735.0,4755.858116
1,35095067,Amoreiras,Campinas,SP,60786.0,6047.275115
2,35095091,Complexo Delta,Campinas,SP,469.0,30.294222


[plus] Comente sobre possíveis dificuldades que você tenha encontrado ao realizar o case.

Uma das dificuldades encontradas foi no dataframe 'concorrentes_df' do arquivo concorrentes.csv, onde existia uma coluna chamada ENDEREÇO, e nessa coluna não existia um padrão muito específico que desse para usar um regex, e  consequentemente extrair (separar) os campo de logradouro, número por exemplo. Sendo necessário fazer um regex para extrair os CEP, e em seguida subtrair os valores de MUNICÍPIO e UF do ENDEREÇO, o que deixou um pouco mais padronizado.